# Harvesting @ MLAI Training - Third Round Overview

At this point in the series of experiments, we're going to try some divergent techniques. I'd like to try some tactics that will improve the fit of the model to continuous use in the live application flow. Right now, the model trains on retrospective data sets and more or less expects to see full sessions. In this experiment, I'm going to try several approaches.

## Roadmap
### Feature engineering and ML algorithms

1. Instead of counting full sessions in the training datasets, we will only take sequences of a maximum N, to see if there's a useful threshold where we could evaluate sliding windows of session history. I'll compare recall at 5, 10, 15, 20 element session subsequences, offset randomly from session start. 
    - Where do the metrics drop off? 
    - Does the approach essentially require full sessions?
    - Can we query partial sessions against a model trained on full sessions?
    - Can we query partial sessions against a model trained on partial sessions?
    - Randomly drop out D% of transactions from every session.
1. Instead of using bag-of-txns, I will try a vector that contains the last N sessions by number, indexed from a transaction dictionary. This will force sequence into the model.
    - Training on the first N transactions:
        - Count N transactions from the beginning of a session.
    - Train on N-gram tiles
    - Train on N-gram shingles
1. Instead of training an XGBoost model, train neural network models that are designed to remember sequences
    - LSTM
    - Convolutional (?)
    
### Software engineering
To this point, I've mostly worked out of a some-what scattered Jupyter notebook pile of global variables, tuning up a few functions here and there to keep scopes clean. In particular, the Sagemaker code is terrible cut-and paste. As I work through the feature engineering & algorithmic roadmap, I also want to drive toward the following engineering goals:

- Establish a consistent data manipulation pipeline that is easily customizable and reentrant, with no global variables
- a set of wrapper functions for the Sagemaker API, so that I can have a simple pipeline where training jobs -> models -> batch transformations or endpoints, without giant string constant parameters that are impossible to edit or sight-check
- Automatically engage a hyperparameter tuning job as desired without changing how the model is implemented or trained.
- Easily training many models at once
- Easily launch many batch transform jobs/endpoints at once.
- Easily display the results of the many jobs at once.
- Particular feature engineering processes glide smoothly from experimental tinkering to code that can be put into production, without throwing everything away. Some of this just falls out of doing everything else better.

# Training Data
The training dataset consists of several hours of raw transaction logs containing activity from all users, with the full collection of harvesting activity from the LiquidTension harvester across two years. All LiquidTension(LT) activity is labeled as 'BadActor' = 1, while all other traffic is assumed to be innocent and labeled as 'BadActor' = 0. Since LiquidTension is currently our only easily-identified single harvester, we need his full range of activity to have a BadActor sessions in proportion to innocent sessions for training to work properly.

The training set includes the following files:

|File       |Contents                             |Rows|
------------|-------------------------------------|----|
|may1.tsv|raw transactions|119474|
|may2.tsv|raw transactions|43608|
|may3.tsv|raw transactions|30844|
|lt-only.tsv|raw transactions for a known attacker|61917|

In all, we have 193926 transactions from "innocent" sessions and 61917 LT sessions. Approximately 32% of transactions are labeled BadActor = 1, giving us a reasonable proportion in both classes.

# Sequence testing phase 1. 

## Warm up
Import standard libraries and prepare the environment.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

import toolz


%matplotlib inline

!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [20]:
# S3 bucket name
bucket = 'sagemaker-mlai-harvesting'

# common column names
bad_col='BadActor'
sess_col='SessionNo'
txn_col='Act'
logtime_col = 'LogTime'

# paths
csv_path = "out"

## Download
Retrieve the datafiles from the project's designated S3 bucket.

In [3]:
s3 = boto3.resource('s3')
b = s3.Bucket('sagemaker-mlai-harvesting')

# b.download_file( 'data/MLAI_ParsedDataSet.tsv', 'data/data.tsv')
b.download_file( "data/MinimalLogs/Minimal_May01.rpt", 'data/may1.tsv')
b.download_file( "data/MinimalLogs/Minimal_May02.rpt", 'data/may2.tsv')
b.download_file( "data/MinimalLogs/Minimal_May03.rpt", 'data/may3.tsv')
b.download_file( "data/MinimalLogs/Minimal_OnlyLT.rpt", 'data/lt-only.tsv')


may1 = pd.read_csv('data/may1.tsv',sep='\t')
may2 = pd.read_csv('data/may2.tsv',sep='\t')
may3 = pd.read_csv('data/may3.tsv',sep='\t')
lt = pd.read_csv('data/lt-only.tsv',sep='\t')



txn = may1.append([may2, may3, lt])
txn[logtime_col] = pd.to_datetime(txn[logtime_col])
# txn[txn[bad_col]==1]

## Data conversion and feature engineering
In real life, a session consists of a series of rows of transactions of different types, and each transaction type records a variable number of additional metadata attributes describing a logged event, for a total of over 30 columns of extracted data. In addition, our tagging process has given each row a BadActor label.

|sessionno|txn id|BadActor|parm1|parm2|...|
|---------|------|--------|-----|-----|---|
|1240|111|0|query string|...|...|
|1240|112|0|meta|...|...|
|2993|301|1|meta|...|...|


In [4]:
# 'Innocent' log entries
txns = pd.DataFrame(np.sort(txn['Act'].unique()))

# Harvesting log entries
lt_txns = pd.DataFrame(np.sort(lt['Act'].unique()))



We drop most of this information, including the temporal sequence of the log entries, and convert each session into a single row of data. Almost all of the columns go away, replaced by counts of transaction types in the session.

|sessionno|BadActor|111|112|113|...|301|302|...|
|---------|--------|---|---|---|---|---|---|---|
|1240|0|1|1|0|...|0|0|...|
|2993|1|0|0|0|...|1|0|...|

# Truncating sessions
Before we flatten the sessions, we're going to truncate them. This may be a better match for the real world, in which at best we will be able to scan sliding windows of transactions with a scaling resumption that we may not scan every event.

We'll try several approaches at once:
- dropping out D% of transactions from every session
- taking only the first N sessions from every session
- taking on the last N transactions from every session
- taking N consecutive transactions from the middle of every session. 
- choosing N transactions as above, but dropping every session without at least N transactions.

In [5]:
def drop_column_groups( txn_c ):
    txn_c.columns = txn_c.columns.droplevel(0)  
    txn_c.rename_axis(None, axis=1).reset_index()
    return txn_c

def get_session_groups( txn ):
    txn_g = txn.groupby(sess_col)
    return txn_g

txng = get_session_groups(txn)
txng

In [6]:
%%time
def drop_pct( df, n = .1 ):
    return df.sample(frac= 1-n)

def first_n( df, n = 5):
    return df.head( n )
    
def last_n( df, n = 5):
    return df.tail( n )


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [7]:
def grappl( fun, parm ):
    return lambda df: df.apply(fun, parm).drop(sess_col, axis=1)


In [8]:
def flatten_txns( txn_log ):
    txn_narrow = txn_log[[sess_col, txn_col,bad_col]]
    txn_pivot = pd.pivot_table(txn_narrow, index=[sess_col,bad_col], columns = [txn_col],aggfunc=[len]).fillna(0)
    txn_pivot.columns = txn_pivot.columns.droplevel(0)           # the pivot table has a two-level index
    txn_flat = txn_pivot.rename_axis(None, axis=1).reset_index() # these two lines get rid of it so we have a simple table
    return txn_flat

In [9]:
def flatten_groups( txn_log ):
    txn_narrow = txn_log[[txn_col,bad_col]]
    txn_pivot = pd.pivot_table(txn_narrow, index=[sess_col,bad_col], columns = [txn_col],aggfunc=[len]).fillna(0)
    txn_pivot.columns = txn_pivot.columns.droplevel(0)           # the pivot table has a two-level index
    txn_flat = txn_pivot.rename_axis(None, axis=1).reset_index() # these two lines get rid of it so we have a simple table
    return txn_flat

In [29]:
jobs = [['drop10', [drop_pct, .1]], 
        ['first5', [first_n, 5]],
        ['last5', [last_n, 5]]
       ]
job_names = [job[0] for job in jobs]
[(name, func, parm) for [name, [func, parm]] in jobs]

[('drop10', <function __main__.drop_pct(df, n=0.1)>, 0.1),
 ('first5', <function __main__.first_n(df, n=5)>, 5),
 ('last5', <function __main__.last_n(df, n=5)>, 5)]

In [11]:
def prep_jobs(df, jobs):
    groups = [[name, grappl(fun, parm)(df)] for [name, [fun, parm]] in jobs]

    flats = [[name,flatten_groups( txn )] for [name,txn] in groups] #.reset_index()
    return flats

In [12]:
%%time

gs = prep_jobs(txng, jobs )


CPU times: user 1min 40s, sys: 535 ms, total: 1min 41s
Wall time: 1min 40s


In [13]:
txngs= gs
[(name,len(df)) for [name,df] in txngs]
# gs[0]

[('drop10', 24114), ('first5', 24112), ('last5', 24112)]

## Producing pools of training and testing data

In order to support simultaneous execution of multiple jobs, this notebook introduces a new scheme for piping data through to models.

The normal flow runs as follows:

input data(s3) -> df's on notebook instance -> train.csv, test.csv, validate.csv on notebook instance -> s3/out/ -> Sagemaker instances

Hardcoding these filenames is fine for playing around in a notebook, but it limits us to one job at a time.

In this approach, every job has a base Name. This name will carry through from S3 into the Sagemaker instances.
Training data files will reside in `<s3bucket>/<key>/out/Name`.

As before, in each `Name` subfolder, we will divide the combined good and bad data pools as follows:
- a training set that the model iterates over during the learning process
- a test set that is used to evaluate the model during training
- a validation set that is kept separate to test the model after training is complete. We need separate test and validate pools in order to make sure that we're not overfitting the model to a single set of test data.

All of these functions are in the FrameSplitter class. 


In [57]:
from importlib import reload
import lib.JupHelper.JupHelper as jh
reload(jh)

<module 'lib.JupHelper.JupHelper' from '/home/ec2-user/SageMaker/mlai-harvesting/lib/JupHelper/JupHelper.py'>

In [58]:
import lib.JupHelper.JupHelper as jh

csv = jh.FrameSplitter( bad_col, [sess_col]) # FrameSplitter only holds onto the definition of the y_col and the x_cols - everything else is passed in

In [61]:
job_csvs = csv.make_all_csvs( gs )           # Drives the whole conversion process - look in the class for other helper methods

In [17]:
csvs = csv.get_all_csv_names(gs)

# Upload to S3 and move into SageMaker

Move all of the current csv's up into S3 for SageMaker, then start configuring jobs.

In [57]:
from importlib import reload
import lib.JupHelper.JupHelper as jh
reload(jh)

<module 'lib.JupHelper.JupHelper' from '/home/ec2-user/SageMaker/mlai-harvesting/lib/JupHelper/JupHelper.py'>

In [212]:
from sagemaker.amazon.amazon_estimator import get_image_uri

import boto3
from time import gmtime, strftime
import time

class XgbHyper:
    def __init__(self):
        self.params = {
            "max_depth":"5",
            "eta":"0.2",
            "gamma":"4",
            "min_child_weight":"6",
            "subsample":"0.7",
            "silent":"0",
            "objective":"binary:logistic",
            "num_round":"50"
        }

class TrainRunner:
    def __init__( self, prefix, region='us-east-1' ):
        '''
        Wrapper object to run multiple 
        '''
        self.prefix = prefix
        self.region = region

        self.running_jobs = []
        self.models = []
        self.sg_client = boto3.client('sagemaker', region_name=region )
        # sagemaker session, role
        self.sagemaker_session = sagemaker.Session()
        self.role = sagemaker.get_execution_role()
    
    def start_jobs(self, jobs, bucket ):
        for (name, csvs) in jobs:
            job_name = self.make_job_name( name )
            print( job_name )
            self.start_training_job( bucket, job_name, csvs )
            self.running_jobs.append([name,job_name])
        return self.running_jobs
    
    def clear_jobs(self):
        self.running_jobs = []
        self.models = []
    
    def start_training_job(self, bucket, job_name, s3_inputs, image='xgboost', instance = [1, "ml.m4.4xlarge", 5 ], hyper=XgbHyper(), verbose=True):
        self.image = image
        self.s3_input = s3_inputs
        self.instance = instance
        self.hyper = hyper
        self.bucket = bucket

        self.container = get_image_uri( self.region, 'xgboost' )
        job_config = self.make_job_config( job_name, s3_inputs[0], s3_inputs[1], s3_inputs[2])
        res = self.launch_training_job( job_config )
        if verbose:
            print( "Started {}. Response: {}".format( job_name, res))
            
        return res
    
    def make_s3_url(self, file):
        return "s3://{}/{}".format(self.bucket, file)
    
    def get_job_status(self, job):
        [name, job_name] = job
        return self.sg_client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    
    def check_jobs_still_running( self ):
        still_running = False
        for job in self.running_jobs:
            status = get_job_status( job )
            if status !='Completed' and status !='Failed':
                still_running = True
                break
        
        return still_running
    
    def map_jobs( self, func ):
        for job in self.running_jobs:
            func( job )
    
    def print_job_status( self, job ):
        [name, job_name] = job
        
        print("{}: {}".format(job, self.get_job_status(job) )  )
        
    def trace_jobs( self ):
        self.map_jobs( self.print_job_status )
                    
    def make_job_name( self, job_name ):
        name = "{}-{}-{}".format( self.prefix, job_name, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
        return name
            
    def launch_training_job( self, job_config ):
        self.sg_client.create_training_job( **job_config )
    
    def wait_for_jobs( self ):
        while True:
            print("Checking job statuses:")
            self.trace_jobs()
            if not self.check_jobs_still_running():
                break
            time.sleep(15)

    def get_model_name(name):
        return name + "-model"
     
    def create_models( self ):
        for [name, job_name] in self.running_jobs:
            info = self.sg_client.describe_training_job(TrainingJobName=job_name)
            model_data = info['ModelArtifacts']['S3ModelArtifacts']
            primary_container = {
                'Image': self.container,
                'ModelDataUrl': model_data
            }
            res = self.sg_client.create_model(
                ModelName = get_model_name(job_name),
                ExecutionRoleArn = role,
                PrimaryContainer = primary_container)
            print(res['ModelArn'])
            self.models.append( res )
            
    def make_job_config(self, job_name, train, test, val ):
        return {
        "AlgorithmSpecification": {
            "TrainingImage": self.container,
            "TrainingInputMode": "File"
        },
        "RoleArn": self.role,
        "OutputDataConfig": {
            "S3OutputPath": os.path.join("s3://", self.bucket, "out", "xgb-class") 
        },
        "ResourceConfig": {
            "InstanceCount": self.instance[0],
            "InstanceType": self.instance[1],
            "VolumeSizeInGB": self.instance[2]
        },
        "TrainingJobName": job_name,
        "HyperParameters": self.hyper.params,        
        "StoppingCondition": {
            "MaxRuntimeInSeconds": 3600
        },
        "InputDataConfig": [
            {
                "ChannelName": "train",
                "DataSource": {
                    "S3DataSource": {
                        "S3DataType": "S3Prefix",
                        "S3Uri": self.make_s3_url( train ), # "s3://sagemaker-mlai-harvesting/out/train.csv" , 
                        "S3DataDistributionType": "FullyReplicated"
                    }
                },
                "ContentType": "text/csv",
                "CompressionType": "None"
            },
            {
                "ChannelName": "test",
                "DataSource": {
                    "S3DataSource": {
                        "S3DataType": "S3Prefix",
                        "S3Uri": self.make_s3_url( test ), # "s3://sagemaker-mlai-harvesting/out/test.csv" , 
                        "S3DataDistributionType": "FullyReplicated"
                    }
                },
                "ContentType": "text/csv",
                "CompressionType": "None"
            },
            {
                "ChannelName": "validation",
                "DataSource": {
                    "S3DataSource": {
                        "S3DataType": "S3Prefix",
                        "S3Uri": self.make_s3_url( val ), # "s3://sagemaker-mlai-harvesting/out/validate.csv" ,
                        "S3DataDistributionType": "FullyReplicated"
                    }
                },
                "ContentType": "text/csv",
                "CompressionType": "None"
            }
        ]
    }


class SageHelper:
    def __init__(self, bucket, image, region='us-east-1'):
        self.bucket = bucket
        self.image = image
        self.container = get_image_uri(region, self.image)
        self.s3_client = boto3.client('s3')
    
    def s3_upload(self, file, s3_path="", verbose = True):
        target = os.path.join(s3_path, file) 
        if verbose:
            print( "Uploading {} to s3://{}/{}".format(file, bucket, target))
        response = self.s3_client.upload_file( file, bucket, target )
        if verbose:
            print( response )
        return response
    
        
   

In [149]:
sh = SageHelper( bucket, 'xgboost' )

In [150]:
for (job, csvs ) in job_csvs:
    print( "Uploading files for {}".format(job))
    for file in csvs:
        sh.s3_upload( file )

Uploading files for drop10
Uploading out/drop10_train.csv to s3://sagemaker-mlai-harvesting/out/drop10_train.csv
None
Uploading out/drop10_test.csv to s3://sagemaker-mlai-harvesting/out/drop10_test.csv
None
Uploading out/drop10_validate.csv to s3://sagemaker-mlai-harvesting/out/drop10_validate.csv
None
Uploading files for first5
Uploading out/first5_train.csv to s3://sagemaker-mlai-harvesting/out/first5_train.csv
None
Uploading out/first5_test.csv to s3://sagemaker-mlai-harvesting/out/first5_test.csv
None
Uploading out/first5_validate.csv to s3://sagemaker-mlai-harvesting/out/first5_validate.csv
None
Uploading files for last5
Uploading out/last5_train.csv to s3://sagemaker-mlai-harvesting/out/last5_train.csv
None
Uploading out/last5_test.csv to s3://sagemaker-mlai-harvesting/out/last5_test.csv
None
Uploading out/last5_validate.csv to s3://sagemaker-mlai-harvesting/out/last5_validate.csv
None


# Prepare and train a model
Boilerplate code mostly copied from Amazon sample code at https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/xgboost_abalone/xgboost_abalone.ipynb, with ample room for improvement.

In [213]:
tr= TrainRunner("harvest-xgb-bc")

In [214]:
%%time

running_jobs = tr.start_jobs(job_csvs, bucket)

tr.wait_for_jobs()

harvest-xgb-bc-drop10-2019-07-08-21-47-39
Started harvest-xgb-bc-drop10-2019-07-08-21-47-39. Response: None
harvest-xgb-bc-first5-2019-07-08-21-47-39
Started harvest-xgb-bc-first5-2019-07-08-21-47-39. Response: None
harvest-xgb-bc-last5-2019-07-08-21-47-45
Started harvest-xgb-bc-last5-2019-07-08-21-47-45. Response: None
Checking job statuses:
['drop10', 'harvest-xgb-bc-drop10-2019-07-08-21-47-39']: InProgress
['first5', 'harvest-xgb-bc-first5-2019-07-08-21-47-39']: InProgress
['last5', 'harvest-xgb-bc-last5-2019-07-08-21-47-45']: InProgress
Checking job statuses:
['drop10', 'harvest-xgb-bc-drop10-2019-07-08-21-47-39']: InProgress
['first5', 'harvest-xgb-bc-first5-2019-07-08-21-47-39']: InProgress
['last5', 'harvest-xgb-bc-last5-2019-07-08-21-47-45']: InProgress
Checking job statuses:
['drop10', 'harvest-xgb-bc-drop10-2019-07-08-21-47-39']: InProgress
['first5', 'harvest-xgb-bc-first5-2019-07-08-21-47-39']: InProgress
['last5', 'harvest-xgb-bc-last5-2019-07-08-21-47-45']: InProgress
Che

# Launching endpoints for trained models.

In a straight-through pipeline, launch endpoints, run tests, collect output, and shut endpoints down.

In [208]:
def get_model_name(name):
    return name + "-model"

for [name, job_name] in tr.running_jobs:
    print(job_name)
    info = tr.sg_client.describe_training_job(TrainingJobName=job_name)
    model_data = info['ModelArtifacts']['S3ModelArtifacts']
    
    primary_container = {
        'Image': tr.container,
        'ModelDataUrl': model_data
    }
    
    res = tr.sg_client.create_model(
        ModelName = get_model_name(job_name),
        ExecutionRoleArn = role,
        PrimaryContainer = primary_container)

    print(res['ModelArn'])



harvest-xgb-bc-drop10-2019-07-08-21-21-10
arn:aws:sagemaker:us-east-1:872344130825:model/harvest-xgb-bc-drop10-2019-07-08-21-21-10-model
harvest-xgb-bc-first5-2019-07-08-21-21-10
arn:aws:sagemaker:us-east-1:872344130825:model/harvest-xgb-bc-first5-2019-07-08-21-21-10-model
harvest-xgb-bc-last5-2019-07-08-21-21-11
arn:aws:sagemaker:us-east-1:872344130825:model/harvest-xgb-bc-last5-2019-07-08-21-21-11-model


In [215]:
tr.create_models()

arn:aws:sagemaker:us-east-1:872344130825:model/harvest-xgb-bc-drop10-2019-07-08-21-47-39-model
arn:aws:sagemaker:us-east-1:872344130825:model/harvest-xgb-bc-first5-2019-07-08-21-47-39-model
arn:aws:sagemaker:us-east-1:872344130825:model/harvest-xgb-bc-last5-2019-07-08-21-47-45-model


In [ ]:
%%time
import boto3
from time import gmtime, strftime

model_name="harvesting-xgboost-binary-class2019-06-21-19-33-25"+ '-model'
print(model_name)

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

In [ ]:
from time import gmtime, strftime

endpoint_config_name = 'Harvest-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


# Launch an endpoint

In [ ]:
%%time
import time

endpoint_name = 'Harvest-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)



# Test the model
Currently, we launch an endpoint to test the model. This endpoint includes a simple web service that takes POST request with rows of or model's X values - columns other than BadActor - and returns a corresponding list of Y values - BadActor predictions.

The endpoint approach is most suitable to interactive use, such as possibly using the model to blacklist a harvesting session as soon as it is identified. For offline analysis, this should be reconfigured to run batch transform jobs instead, which are cheaper to run and more streamlined to invoke.

In [ ]:
runtime_client = boto3.client('runtime.sagemaker', region_name=region)

import json
from itertools import islice
import math
import struct

!head -10000 out/test.csv > out/single-test.csv

file_name = 'out/single-test.csv' 

# file_name = "out/may8.csv"


csv = pd.read_csv(file_name, header=None)
csv.columns
label = csv[0]
csv = csv.drop(0,axis=1)

single = "out/single.csv"

csv.to_csv(path_or_buf=single, header=False, index=False)

with open(single, 'r') as f:
    payload = f.read().strip()

In [ ]:
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read()
result = result.decode("utf-8")
result = result.split(',')
result = [round(float(i)) for i in result]


# Compute the confusion metrics

A confusion matrix describes the proportions of true and false positives and negatives, together with some derived metrics.

In [ ]:
comp = pd.concat( [label, pd.DataFrame(result)], axis = 1)
comp.columns =["label",'prediction']

label_positive = comp['label'] == 1
predict_positive = comp['prediction'] == 1

tp = len( comp[label_positive & predict_positive])
fp = len( comp[~label_positive & predict_positive])
tn = len( comp[~label_positive & ~predict_positive])
fn = len( comp[label_positive & ~predict_positive])
m = len(comp)

accuracy = (tp+tn)/m
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("accuracy: {} precision: {} recall {}".format(accuracy, precision,recall))

In [ ]:
tp,fp,tn,fn, len(comp)

# V1 results
accuracy: 0.9657853810264385 precision: 1.0 recall 0.8360655737704918

|tp|fp|tn|fn|len(comp)|
|--|--|--|--|---------|
|1683| 0| 7632| 330|9645|

The very first time we ran the model, we achieved strikingly successful rates of harvesting identification.
The most significant number here is the recall of 84%, meaning that we successfully identified 84% of all harvesting sessions by looking only at counts of transaction types.


# V2 results, with txns + wait stats:
accuracy: 0.968754320475598 precision: 0.9665454545454546 recall 0.8807157057654076

|tp|fp|tn|fn|len(comp)|
|--|--|--|--|---------|
|1329| 46| 5678| 180|7233|

In the second run, we see an increased recall, at the price of a small number of false positives. 

# Next steps

## Further investigating the data 

We had additional ideas for modeling the data while staying in this bag-of-transaction technique.
1. Try some hyperparameter tuning to seem if the success rates can be trivially improved.
1. Enrich the training data set in various ways - add colums to summarize total session time, average time/request, and so on.
1. Perform some clustering analysis to try to identify common patterns of behavior other than LT. This may reveal the presence of other kinds of harvesting.

## Qualifying the approach
Can we use this approach to identify and blacklist harvesting sessions as they occur? Some notes:
1. The approach must be resilient to easy efforts to evade. Does the accuracy of the identification drop if the attacker makes minor changes to his workflow?
1. How long does it take to identify an attacker in real time? 
    1. Do we gain certainty soon enough to stop an attacker before he's done what he came to do?
    2. Can we tag sessions accurately after the first N log entries, for instance?
    
## Designing an implemetation
Design an architecture for identifying and intercepting harvesting activity in real time. Confirm data sources, manage impact to usage latency, model costs and ROI.

In today's world, it would be less effective to perform real-time analysis on AWS, since all of our current content usage is on-prem. The algorithm used here, XGBoost, is performant on commodity hardware, so we may be able to run on standard VMs.

In real-time analysis, we will face a stream of events from interleaved sessions. We will have to demultiplex these into individual event streams both for training and for prediction, implying some kind of windowing to capture and send sets of log entries as partial sessions. It's not clear how big the impact of this windowing will be on the accuracy of the models.

# Other analytical techniques
While this algorithm seems promising, we're throwing away a huge amount of intelligence before we start training, in the name of simplicity. We can evaluate what kind of gains we could achieve through more advanced techniques:
- Stateful models like LSTM or CNN
- more 
